### Clear Sky

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import bisect
import datetime
from scipy import interpolate
from matplotlib.dates import DateFormatter

# User defined functions
from import_WS import *
from load_data_from_csv import *

### Import LUZ Data

In [ ]:
luz_csv = r'irradiation_luz_2017_2018.csv'
